In [ ]:
import pandas as pd
import cvxpy as cp

# File path will be different on another device and must be replaced with the correct file path.
students = pd.read_csv("C:/Users/Morgan Eads/Documents/ISE 3230/Sample Data (for scheduling) - Sheet1.csv")
print(students.info())